In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/diabetes.csv")

In [6]:
df.shape

(768, 9)

In [7]:
df.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

## Separate X & Y

In [8]:
X = df.drop("Outcome", axis=1)
Y = df['Outcome']

In [9]:
X.shape, Y.shape

((768, 8), (768,))

In [10]:
X.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
dtype: int64

In [11]:
X.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
dtype: object

In [12]:
X_ohe = pd.get_dummies(X)

In [13]:
X_ohe.shape

(768, 8)

In [14]:
X_ohe.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
dtype: object

## Format the Target Y

In [15]:
Y.dtype

dtype('int64')

In [16]:
Y.value_counts()

Outcome
0    500
1    268
Name: count, dtype: int64

In [17]:
Y.loc[Y == 0] = 'Healthy'
Y.loc[Y == 1] = 'Diabetic'

<ipython-input-17-27f493855730>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y.loc[Y == 0] = 'Healthy'


In [18]:
Y.unique()

array(['Diabetic', 'Healthy'], dtype=object)

In [19]:
Y.dtype

dtype('O')

## Train Test Split

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X_ohe, Y,
                                                    test_size=0.3,
                                                    random_state = 7,
                                                    stratify = Y)

In [22]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((537, 8), (231, 8), (537,), (231,))

## **Feature Selection**

In [23]:
from sklearn.decomposition import PCA

In [24]:
from sklearn.tree import DecisionTreeClassifier

In [25]:
pca = PCA(n_components=0.99, random_state=7)

In [26]:
pca.fit(X_train)

PCA(n_components=0.99, random_state=7)

In [27]:
pca.components_

array([[-1.37732832e-03,  9.51139842e-02,  1.86451108e-02,
         6.08687285e-02,  9.93322216e-01,  1.43998971e-02,
         6.88260289e-04, -1.39145429e-03],
       [-2.80834702e-02, -9.69385346e-01, -1.22604296e-01,
         9.90740633e-02,  8.94326849e-02, -4.40725646e-02,
        -1.01547507e-03, -1.57207694e-01],
       [-2.94257537e-02,  1.19000040e-01, -9.34629290e-01,
        -2.84012965e-01,  2.49410889e-02, -1.11440583e-01,
        -6.36338300e-04, -1.33177984e-01],
       [-4.00376841e-02,  1.53874457e-01, -2.28888920e-01,
         8.72630175e-01, -6.78217271e-02,  2.35104514e-01,
         3.24408809e-03, -3.17744603e-01],
       [ 1.54378642e-01, -9.64937972e-02, -2.38190443e-01,
         2.77827381e-01, -2.64637666e-03,  5.66244058e-02,
         2.04099601e-03,  9.10887451e-01]])

In [28]:
pca.n_components_

5

In [29]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
dt = DecisionTreeClassifier(criterion = 'entropy', random_state=7, class_weight='balanced')

In [32]:
dt.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       random_state=7)

In [33]:
dt.feature_importances_

array([0.05200389, 0.27625148, 0.06044403, 0.04980409, 0.06756502,
       0.20965836, 0.09695069, 0.18732243])

In [34]:
feature_tup = tuple(zip(X_train.columns, dt.feature_importances_))

In [35]:
sorted(feature_tup, key = lambda x:-x[1])

[('Glucose', 0.2762514846814094),
 ('BMI', 0.20965836349032377),
 ('Age', 0.187322433955868),
 ('DiabetesPedigreeFunction', 0.09695069387612644),
 ('Insulin', 0.06756501826911719),
 ('BloodPressure', 0.06044402640182533),
 ('Pregnancies', 0.05200389015850862),
 ('SkinThickness', 0.04980408916682127)]

## Library mwthod for selecting best features using Decision Tree

In [36]:
from sklearn.feature_selection import SelectFromModel

In [37]:
from sklearn.tree import DecisionTreeClassifier

In [38]:
sel = SelectFromModel(DecisionTreeClassifier(criterion = 'entropy',
                                             random_state=7,
                                             class_weight='balanced'),
                                             threshold = "mean")

In [39]:
sel.fit(X_train, Y_train)

SelectFromModel(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                 criterion='entropy',
                                                 random_state=7),
                threshold='mean')

In [40]:
sel.get_feature_names_out

<bound method SelectorMixin.get_feature_names_out of SelectFromModel(estimator=DecisionTreeClassifier(class_weight='balanced',
                                                 criterion='entropy',
                                                 random_state=7),
                threshold='mean')>

In [41]:
X_train_DT = sel.transform(X_train)
X_test_DT = sel.transform(X_test)

In [42]:
sel.get_feature_names_out(X_train.columns)

array(['Glucose', 'BMI', 'Age'], dtype=object)

In [43]:
sel.estimator_.feature_importances_

array([0.05200389, 0.27625148, 0.06044403, 0.04980409, 0.06756502,
       0.20965836, 0.09695069, 0.18732243])

In [44]:
X_train.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')